In [1]:
from seeq import spy #imports the spy connector to allow the connection to Seeq
import pandas as pd
from seeq.spy.assets import Asset
#To automatically run all lines of code, hit the "restart the kernel, then re-run the whole notebook (with dialog)" button (double right arrow)

# 1. Generate Workbook and Search Path

In [2]:
workbook_name = 'Brintnell Prod1 Report'#Edit Required-> change name to desired Seeq Workbook to access
signal_search_name = 'Liquid Flow Rate'#Edit Required-> enter the 1st signal required in your formula
search_path = 'Brintnell / Pelican Wells >> Producers >> * >> * >> *'# Edit Required-> Adjust based on Producers or Injector path you want the code to reference. * means ALL Assets in the corresponding hierarchy level

In [3]:
rate_result_df= spy.search(
    {"Name": signal_search_name, 
     "Path": search_path
    }, quiet = True).drop_duplicates(subset = ["Asset"], keep = "first")
#search all Producers (or Injectors if changed to "Injectors") in area for the signal Liquid Flow rate (signal_search_name)

In [4]:
def signal_search_check(search_result_df, signal_check_name):
    search_assets = search_result_df['Asset'].tolist()
    search_path = search_result_df['Path'].tolist()
    #Make a list of the asset & paths found for initail searched signals (signal_search_name)
    
    search_dict = {'Asset': search_assets} #generates a dictionary of all the Assets in search_results
    search_check_df = pd.DataFrame(search_dict)
    search_check_df['Name'] = '/^' + signal_check_name + '$/'
    search_check_df['Path'] = search_path
    return spy.search(search_check_df, quiet = True).drop_duplicates(subset = ["Asset"], keep = "first")
    #Search the list from "Liquid FLow Rates" signals and removes any duplicates (this step assists if there are multiple signals generated in the hiearchy due to an upload error, keeps the first signal only)

# 2. Create signal searches

In [5]:
flow_rate_search_df = signal_search_check(search_result_df = rate_result_df , signal_check_name = 'Liquid Flow Rate')#Edit Required-> change the signal_check_name to the signal name required in your code. Update rod_search_df to the unique name required for your code
bsw_search_df = signal_search_check(search_result_df = rate_result_df , signal_check_name = 'Test BSW')#Edit Required-> change the signal_check_name to the signal name required in your code. Update torque_search_df to the unique name required for your code
rod_search_df = signal_search_check(search_result_df = rate_result_df , signal_check_name = 'Rod Speed')#Edit Required-> change the signal_check_name to the signal name required in your code. Update rod_search_df to the unique name required for your code
eff_search_df = signal_search_check(search_result_df = rate_result_df , signal_check_name = 'SAM_PMP_EFF')#Edit Required-> change the signal_check_name to the signal name required in your code. Update eff_search_df to the unique name required for your code
maxspeed_search_df = signal_search_check(search_result_df = rate_result_df , signal_check_name = 'SAM_MAX_SPEED')#Edit Required-> change the signal_check_name to the signal name required in your code. Update eff_search_df to the unique name required for your code
minspeed_search_df = signal_search_check(search_result_df = rate_result_df , signal_check_name = 'SAM_MIN_SPEED')#Edit Required-> change the signal_check_name to the signal name required in your code. Update eff_search_df to the unique name required for your code

#need to list all of your signals here. Either Add/Delete rows if additional/fewer signals are required, respectively
#The signal_name_check 'variable' should be the exact name pulled from your Seeq workbook (ie. Rod Speed,JTF, Polymer Injection Rate etc.)


# 3. Create 1 Main Report

In [6]:
meta_data_df = flow_rate_search_df.append(bsw_search_df, ignore_index = True)#Edit Required-> update the "*_search_df" changed in Step #2
meta_data_df = meta_data_df.append(rod_search_df, ignore_index = True)#Edit Required-> update the "*_search_df" changed in Step #2
meta_data_df = meta_data_df.append(eff_search_df, ignore_index = True)#Edit Required-> update the "*_search_df" changed in Step #2
meta_data_df = meta_data_df.append(maxspeed_search_df, ignore_index = True)#Edit Required-> update the "*_search_df" changed in Step #2
meta_data_df = meta_data_df.append(minspeed_search_df, ignore_index = True)#Edit Required-> update the "*_search_df" changed in Step #2

meta_data_df =  meta_data_df.assign(Field = meta_data_df['Path'].str.split(">>").str[2].str.strip())
meta_data_df =  meta_data_df.assign(Pad = meta_data_df['Path'].str.split(">>").str[3].str.strip())
meta_data_df['Build Path'] = 'POC Prod Report'#Edit Required-> change 'NVA Variance Report' to desired Path Name for your code (this will be linked to WF2.)
meta_data_df['Build Asset'] = meta_data_df['Field']
#This step merges each individual signal report into 1 main report 

In [7]:
meta_data_df
#displays the main report

,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Field,Pad,Build Path,Build Asset
0,994C3934-A22F-4043-8D1B-55AB611DD659,Brintnell / Pelican Wells >> Producers >> Cent...,100/13-20-081-22W4,Liquid Flow Rate,BP01 100/13-20-081-22W4 Liquid Flow Rate,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP01,POC Prod Report,Central Brintnell
1,E558CF28-C46B-42AA-AA36-9DDC51D7E1C9,Brintnell / Pelican Wells >> Producers >> Cent...,103/05A-21-081-22W4,Liquid Flow Rate,BP02 103/05A-21-081-22W4 Liquid Flow Rate,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP02,POC Prod Report,Central Brintnell
2,E6E99BDE-03F7-4897-8C60-1E462D423E8F,Brintnell / Pelican Wells >> Producers >> Cent...,104/09D-20-081-22W4,Liquid Flow Rate,BP02 104/09D-20-081-22W4 Liquid Flow Rate,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP02,POC Prod Report,Central Brintnell
3,2230B928-BC4F-4CAE-87E3-9E8DE2BD003E,Brintnell / Pelican Wells >> Producers >> Cent...,102/12D-08-081-22W4,Liquid Flow Rate,BP03 102/12D-08-081-22W4 Liquid Flow Rate,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP03,POC Prod Report,Central Brintnell
4,168B79BC-863F-4D7C-A312-F611E8365B4A,Brintnell / Pelican Wells >> Producers >> Cent...,102/13C-08-081-22W4,Liquid Flow Rate,BP03 102/13C-08-081-22W4 Liquid Flow Rate,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP03,POC Prod Report,Central Brintnell
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5352,242537F3-7AD8-40BA-ADBA-0226240BDF31,Brintnell / Pelican Wells >> Producers >> Sout...,103/16D-07-081-22W4,SAM_MIN_SPEED,WB42 103/16D-07-081-22W4 SAM_MIN_SPEED,CalculatedSignal,NaN,Well Structure,False,South Brintnell,WB42,POC Prod Report,South Brintnell
5353,256BF81F-A627-4650-8434-1069E74706BE,Brintnell / Pelican Wells >> Producers >> Sout...,100/03-32-080-22W4,SAM_MIN_SPEED,WB43 100/03-32-080-22W4 SAM_MIN_SPEED,CalculatedSignal,NaN,Well Structure,False,South Brintnell,WB43,POC Prod Report,South Brintnell
5354,C338E178-1E9A-41ED-B4B5-7C0A54AA0D89,Brintnell / Pelican Wells >> Producers >> Sout...,100/01-15-081-23W4,SAM_MIN_SPEED,WB46 100/01-15-081-23W4 SAM_MIN_SPEED,CalculatedSignal,NaN,Well Structure,False,South Brintnell,WB46,POC Prod Report,South Brintnell
5355,A184B825-0F35-4C87-8F04-D67B920E9D0A,Brintnell / Pelican Wells >> Producers >> Sout...,100/11D-01-081-23W4,SAM_MIN_SPEED,WB46 100/11D-01-081-23W4 SAM_MIN_SPEED,CalculatedSignal,NaN,Well Structure,False,South Brintnell,WB46,POC Prod Report,South Brintnell


# 4. Create Formulas for workflow

In [8]:
#Copy/Paste all formulas from your Seeq Workbook (Seeq Workbook-> Variable-> Item Properties-> Formula)
#Assign a name to the formula (ie. variance_formula=...)

#generated oil variance between last 3days and 30days
oilvariance_formula = """
$oilrate_forumla= ((100-$tb)/100)*($lfr)
$threeday_formula=$oilrate_forumla.movingAverageFilter(30min,3d)
$thirthyday_forumla=$oilrate_forumla.movingAverageFilter(150min,30d)
$orvar= $threeday_formula-$thirthyday_forumla
$orvar
"""
#generated Fluid variance between last 3days and 30days
fluid_variance_formula = """
$twohr_formula= $lfr.movingAverageFilter(30min,3d)
$twentyfourhr_formula= $lfr.movingAverageFilter(150min,30d)
$frcond_20 = $twohr_formula-$twentyfourhr_formula
$frcond_20
"""
#generates BSW cahnge >20%
bsw_variance_forumla = """
$period=periods(24h,30min)
$bsw_change=$tb.aggregate(range(), $period, endKey(), 30.016666666666666min)
$x=$bsw_change > 20
$x
"""
#generates when pump eff >100% and rpm>10rpm
pump_eff_abv100_formula = """
$pumpeff= $pmpeff > 100
$rpmvar= $rpm > 10
$pumpeff_100= $pumpeff.setMaximumDuration(24h).aggregate(count(), $rpmvar.setMaximumDuration(24h), durationKey())
$pumpeff_100
"""

#generates when rod speed within 2rpm of Max Speed
poc_max_speed_formula= """
$max_speed = $rs-2
$deviation1 = $rw - $max_speed
$results1 = valueSearch($deviation1, 1day, isGreaterThan(0), 0min, true, isLessThan(0), 0min, true)
$results1
"""
#generates when rod speed within 2rpm of Max Speed
poc_min_speed_formula= """
$min_speed = $rm+2
$deviation2 = $min_speed- $rw 
$results2 = valueSearch($deviation2, 1day, isGreaterThan(0), 0min, true, isLessThan(0), 0min, true)
$results2
"""

#variance_formula = """

#$twohr_formula= $d.movingAverageFilter(30min,2hr)
#$twentyfourhr_formula= $d.movingAverageFilter(30min,24hr)
#$frcond_20 = ((($twohr_formula-$twentyfourhr_formula)/$twentyfourhr_formula)*100).setUnits('%')
#$frcond_20
#"""

#mvg_avg_formula = """
#$frcond_20 = $v.movingAverageFilter(30min,2hr)
#$frcond_20
#"""

# 5. Create Defintions 

In [9]:
#definition or recipe for custom hierarchy
class Report(Asset):
    @Asset.Component()
    def Pads(self, metadata):
        return self.build_components(template = Pad, metadata = metadata, column_name = 'Pad')
                                     
class Pad(Asset):
    @Asset.Component()
    def Wells(self, metadata):
        return self.build_components(template = Well, metadata = metadata, column_name = 'Asset') 

class Well(Asset):
    @Asset.Attribute()
    def Liquid_Flow_Rate(self, metadata):#Edit Required-> Adjust name as required (ie. Liquid_Flow_Rate)
        return metadata[metadata['Name'].str.match('Liquid Flow Rate')]#Edit Required-> enter signal_search_name from Step #1
    
    @Asset.Attribute()
    def Test_BSW(self, metadata):#Edit Required-> Adjust name as required (ie. Liquid_Flow_Rate)
        return metadata[metadata['Name'].str.match('Test BSW')]#Edit Required-> enter signal_search_name from Step #1
    
    @Asset.Attribute()
    def RPM_Speed(self, metadata):#Edit Required-> Adjust name as required (ie. Liquid_Flow_Rate)
        return metadata[metadata['Name'].str.match('Rod Speed')]#Edit Required-> enter signal_search_name from Step #1
    
    @Asset.Attribute()
    def Pump_Eff(self, metadata):#Edit Required-> Adjust name as required (ie. Liquid_Flow_Rate)
        return metadata[metadata['Name'].str.match('SAM_PMP_EFF')]#Edit Required-> enter signal_search_name from Step #1
    
    @Asset.Attribute()
    def MAX_SPEED(self, metadata):#Edit Required-> Adjust name as required (ie. Liquid_Flow_Rate)
        return metadata[metadata['Name'].str.match('SAM_MAX_SPEED')]#Edit Required-> enter signal_search_name from Step #1
    
    @Asset.Attribute()
    def MIN_SPEED(self, metadata):#Edit Required-> Adjust name as required (ie. Liquid_Flow_Rate)
        return metadata[metadata['Name'].str.match('SAM_MIN_SPEED')]#Edit Required-> enter signal_search_name from Step #1
    
    @Asset.Attribute()
    def Oil_Rate_Variance(self, metadata):#Edit Required-> Adjust name as required (ie. Flow_Rate_Variance)
        return {
            'Type': 'Signal', #Edit Required-> Keep as Signal if it's a variable, change to condition if using a Tool Condition
            'Formula': oilvariance_formula, #Edit Required-> Update to name for the formula used in Step #4
            'Formula Parameters': {
                '$lfr': self.Liquid_Flow_Rate(metadata),   #Edit Required-> ensure the $letter is same as Step #4 
                '$tb': self.Test_BSW(metadata)   #Edit Required-> ensure the $letter is same as Step #4 
            }           
            
        }

    @Asset.Attribute()
    def Flow_Rate_Variance(self, metadata):#Edit Required-> Adjust name as required (ie. Flow_Rate_Variance)
        return {
            'Type': 'Signal', #Edit Required-> Keep as Signal if it's a variable, change to condition if using a Tool Condition
            'Formula': fluid_variance_formula, #Edit Required-> Update to name for the formula used in Step #4
            'Formula Parameters': {
                '$lfr': self.Liquid_Flow_Rate(metadata)   #Edit Required-> ensure the $letter is same as Step #4            
            }           
            
        }
    
    @Asset.Attribute()
    def BSW_Variance(self, metadata):#Edit Required-> Adjust name as required (ie. Flow_Rate_Variance)
        return {
            'Type': 'Condition', #Edit Required-> Keep as Signal if it's a variable, change to condition if using a Tool Condition
            'Formula': bsw_variance_forumla, #Edit Required-> Update to name for the formula used in Step #4
            'Formula Parameters': {
                '$tb': self.Test_BSW(metadata)   #Edit Required-> ensure the $letter is same as Step #4            
            }           
            
        }
    
    @Asset.Attribute()
    def Pump_Eff_abv100(self, metadata):#Edit Required-> Adjust name as required (ie. Flow_Rate_Variance)
        return {
            'Type': 'Signal', #Edit Required-> Keep as Signal if it's a variable, change to condition if using a Tool Condition
            'Formula': pump_eff_abv100_formula, #Edit Required-> Update to name for the formula used in Step #4
            'Formula Parameters': {
                '$pmpeff': self.Pump_Eff(metadata),   #Edit Required-> ensure the $letter is same as Step #4 
                '$rpm': self.RPM_Speed(metadata)   #Edit Required-> ensure the $letter is same as Step #4  
            }           
            
        }

    @Asset.Attribute()
    def POC_Max_Speed(self, metadata):#Edit Required-> Adjust name as required (ie. Flow_Rate_Variance)
        return {
            'Type': 'Condition', #Edit Required-> Keep as Signal if it's a variable, change to condition if using a Tool Condition
            'Formula': poc_max_speed_formula, #Edit Required-> Update to name for the formula used in Step #4
            'Formula Parameters': {
                '$rs': self.MAX_SPEED(metadata),   #Edit Required-> ensure the $letter is same as Step #4 
                '$rw': self.RPM_Speed(metadata)   #Edit Required-> ensure the $letter is same as Step #4  
            }           
            
        }
    
    @Asset.Attribute()
    def POC_Min_Speed(self, metadata):#Edit Required-> Adjust name as required (ie. Flow_Rate_Variance)
        return {
            'Type': 'Condition', #Edit Required-> Keep as Signal if it's a variable, change to condition if using a Tool Condition
            'Formula': poc_min_speed_formula, #Edit Required-> Update to name for the formula used in Step #4
            'Formula Parameters': {
                '$rm': self.MIN_SPEED(metadata),   #Edit Required-> ensure the $letter is same as Step #4 
                '$rw': self.RPM_Speed(metadata)   #Edit Required-> ensure the $letter is same as Step #4  
            }           
            
        }
    
    #Repeat the @ Liquid_Flow_Rate edits (just above) for the below def based on the signals/formulas your wanted to generate
    
    #@Asset.Attribute()
    #def Torque(self, metadata):
        #return metadata[metadata['Name'].str.match('Torque')]

    #@Asset.Attribute()
    #def Torque_Variance(self, metadata):
        #return {
            #'Type': 'Signal',
            #'Formula': variance_formula,
            #'Formula Parameters': {
                #'$d': self.Torque(metadata)                
            #}           
            
        #}
   


In [10]:
#build asset tree from "Report" definition
build_df = spy.assets.build(Report, meta_data_df)

In [11]:
build_df

,ID,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Field,Pad,Referenced Name,Referenced Path,Referenced Asset,Reference,Name,Asset,Asset Object,Path,Template,Build Result,Formula,Formula Parameters
0,0F0F6641-0A1B-4F83-BA04-E952776D0E2D,BP01 100/13-20-081-22W4 Test BSW,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP01,Test BSW,Brintnell / Pelican Wells >> Producers >> Cent...,100/13-20-081-22W4,True,Test BSW,100/13-20-081-22W4,POC Prod Report >> Central Brintnell >> BP01 >...,POC Prod Report >> Central Brintnell >> BP01,Well,Success,NaN,NaN
1,NaN,NaN,Condition,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BSW Variance,100/13-20-081-22W4,POC Prod Report >> Central Brintnell >> BP01 >...,POC Prod Report >> Central Brintnell >> BP01,Well,Success,"$period=periods(24h,30min)\n$bsw_change=$tb.ag...",{'$tb': {'ID': '0F0F6641-0A1B-4F83-BA04-E95277...
2,994C3934-A22F-4043-8D1B-55AB611DD659,BP01 100/13-20-081-22W4 Liquid Flow Rate,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP01,Liquid Flow Rate,Brintnell / Pelican Wells >> Producers >> Cent...,100/13-20-081-22W4,True,Liquid Flow Rate,100/13-20-081-22W4,POC Prod Report >> Central Brintnell >> BP01 >...,POC Prod Report >> Central Brintnell >> BP01,Well,Success,NaN,NaN
3,NaN,NaN,Signal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Flow Rate Variance,100/13-20-081-22W4,POC Prod Report >> Central Brintnell >> BP01 >...,POC Prod Report >> Central Brintnell >> BP01,Well,Success,$twohr_formula= $lfr.movingAverageFilter(30min...,{'$lfr': {'ID': '994C3934-A22F-4043-8D1B-55AB6...
4,8F26FBAE-D6D2-48C5-BF66-66D48B354EA1,BP01 100/13-20-081-22W4 SAM_MAX_SPEED,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP01,SAM_MAX_SPEED,Brintnell / Pelican Wells >> Producers >> Cent...,100/13-20-081-22W4,True,MAX SPEED,100/13-20-081-22W4,POC Prod Report >> Central Brintnell >> BP01 >...,POC Prod Report >> Central Brintnell >> BP01,Well,Success,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,NaN,NaN,Condition,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POC Min Speed,100/15C-01-081-23W4,POC Prod Report >> South Brintnell >> WB46 >> ...,POC Prod Report >> South Brintnell >> WB46,Well,Success,$min_speed = $rm+2\n$deviation2 = $min_speed- ...,{'$rm': {'ID': '9780E9BA-63B1-443F-A421-48D755...
11445,CDC47B4A-FB40-4BB8-8C46-A363BEABC733,WB46 100/15C-01-081-23W4 SAM_PMP_EFF,CalculatedSignal,NaN,Well Structure,False,South Brintnell,WB46,SAM_PMP_EFF,Brintnell / Pelican Wells >> Producers >> Sout...,100/15C-01-081-23W4,True,Pump Eff,100/15C-01-081-23W4,POC Prod Report >> South Brintnell >> WB46 >> ...,POC Prod Report >> South Brintnell >> WB46,Well,Success,NaN,NaN
11446,NaN,NaN,Signal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pump Eff abv100,100/15C-01-081-23W4,POC Prod Report >> South Brintnell >> WB46 >> ...,POC Prod Report >> South Brintnell >> WB46,Well,Success,$pumpeff= $pmpeff > 100\n$rpmvar= $rpm > 10\n$...,{'$pmpeff': {'ID': 'CDC47B4A-FB40-4BB8-8C46-A3...
11447,NaN,NaN,Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100/15C-01-081-23W4,100/15C-01-081-23W4,POC Prod Report >> South Brintnell >> WB46 >> ...,POC Prod Report >> South Brintnell >> WB46,Well,Success,NaN,NaN


In [12]:
#remove assets with no type i.e wells without SAM_PUMP_EFF signal will be excluded
build_df = build_df[build_df['Type'].notna()].reset_index()

In [13]:
#check Build Result for unsucessful rows
build_df[build_df['Build Result'] != 'Success'][['Type', 'Asset', 'Path', 'Build Result']]

,Type,Asset,Path,Build Result


In [14]:
build_df[['ID','Type','Asset', 'Name', 'Build Result','Path', 'Asset Object', 'Template']]
#builds the report (items) that will be presented in your report

,ID,Type,Asset,Name,Build Result,Path,Asset Object,Template
0,0F0F6641-0A1B-4F83-BA04-E952776D0E2D,CalculatedSignal,100/13-20-081-22W4,Test BSW,Success,POC Prod Report >> Central Brintnell >> BP01,POC Prod Report >> Central Brintnell >> BP01 >...,Well
1,NaN,Condition,100/13-20-081-22W4,BSW Variance,Success,POC Prod Report >> Central Brintnell >> BP01,POC Prod Report >> Central Brintnell >> BP01 >...,Well
2,994C3934-A22F-4043-8D1B-55AB611DD659,CalculatedSignal,100/13-20-081-22W4,Liquid Flow Rate,Success,POC Prod Report >> Central Brintnell >> BP01,POC Prod Report >> Central Brintnell >> BP01 >...,Well
3,NaN,Signal,100/13-20-081-22W4,Flow Rate Variance,Success,POC Prod Report >> Central Brintnell >> BP01,POC Prod Report >> Central Brintnell >> BP01 >...,Well
4,8F26FBAE-D6D2-48C5-BF66-66D48B354EA1,CalculatedSignal,100/13-20-081-22W4,MAX SPEED,Success,POC Prod Report >> Central Brintnell >> BP01,POC Prod Report >> Central Brintnell >> BP01 >...,Well
...,...,...,...,...,...,...,...,...
11444,NaN,Condition,100/15C-01-081-23W4,POC Min Speed,Success,POC Prod Report >> South Brintnell >> WB46,POC Prod Report >> South Brintnell >> WB46 >> ...,Well
11445,CDC47B4A-FB40-4BB8-8C46-A363BEABC733,CalculatedSignal,100/15C-01-081-23W4,Pump Eff,Success,POC Prod Report >> South Brintnell >> WB46,POC Prod Report >> South Brintnell >> WB46 >> ...,Well
11446,NaN,Signal,100/15C-01-081-23W4,Pump Eff abv100,Success,POC Prod Report >> South Brintnell >> WB46,POC Prod Report >> South Brintnell >> WB46 >> ...,Well
11447,NaN,Asset,100/15C-01-081-23W4,100/15C-01-081-23W4,Success,POC Prod Report >> South Brintnell >> WB46,POC Prod Report >> South Brintnell >> WB46 >> ...,Well


In [15]:
#push asset tree to workbook defined by workbook_name variable assigned in Step #1
spy.push(metadata=build_df, workbook = workbook_name)

,index,ID,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Field,Pad,Referenced Name,...,Asset Object,Path,Template,Build Result,Formula,Formula Parameters,Datasource Class,Datasource ID,Data ID,Push Result
0,0,F06AD618-C38E-40AA-ADED-87A8CB8A0E88,BP01 100/13-20-081-22W4 Test BSW,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP01,Test BSW,...,POC Prod Report >> Central Brintnell >> BP01 >...,POC Prod Report >> Central Brintnell >> BP01,Well,Success,NaN,[signal=0F0F6641-0A1B-4F83-BA04-E952776D0E2D],Seeq Data Lab,Seeq Data Lab,[7C166C13-9C0E-4C9D-8721-D1B4F894C13F] {Signal...,Success
1,1,5E49A3B6-1590-4AE1-AC9B-03A54544CA0F,NaN,CalculatedCondition,NaN,NaN,NaN,NaN,NaN,NaN,...,POC Prod Report >> Central Brintnell >> BP01 >...,POC Prod Report >> Central Brintnell >> BP01,Well,Success,"$period=periods(24h,30min)\n$bsw_change=$tb.ag...",[tb=F06AD618-C38E-40AA-ADED-87A8CB8A0E88],Seeq Data Lab,Seeq Data Lab,[7C166C13-9C0E-4C9D-8721-D1B4F894C13F] {Condit...,Success
2,2,82A31337-A06E-4974-B63D-AAE886DD4941,BP01 100/13-20-081-22W4 Liquid Flow Rate,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP01,Liquid Flow Rate,...,POC Prod Report >> Central Brintnell >> BP01 >...,POC Prod Report >> Central Brintnell >> BP01,Well,Success,NaN,[signal=994C3934-A22F-4043-8D1B-55AB611DD659],Seeq Data Lab,Seeq Data Lab,[7C166C13-9C0E-4C9D-8721-D1B4F894C13F] {Signal...,Success
3,3,370F43E3-DE68-4DFE-8CDC-5657024005F8,NaN,CalculatedSignal,NaN,NaN,NaN,NaN,NaN,NaN,...,POC Prod Report >> Central Brintnell >> BP01 >...,POC Prod Report >> Central Brintnell >> BP01,Well,Success,$twohr_formula= $lfr.movingAverageFilter(30min...,[lfr=82A31337-A06E-4974-B63D-AAE886DD4941],Seeq Data Lab,Seeq Data Lab,[7C166C13-9C0E-4C9D-8721-D1B4F894C13F] {Signal...,Success
4,4,EE24F639-402B-45CE-9676-28AF5E086CAE,BP01 100/13-20-081-22W4 SAM_MAX_SPEED,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP01,SAM_MAX_SPEED,...,POC Prod Report >> Central Brintnell >> BP01 >...,POC Prod Report >> Central Brintnell >> BP01,Well,Success,NaN,[signal=8F26FBAE-D6D2-48C5-BF66-66D48B354EA1],Seeq Data Lab,Seeq Data Lab,[7C166C13-9C0E-4C9D-8721-D1B4F894C13F] {Signal...,Success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,11444,46F13246-B2D9-4322-97DC-F1D6930D9CE2,NaN,CalculatedCondition,NaN,NaN,NaN,NaN,NaN,NaN,...,POC Prod Report >> South Brintnell >> WB46 >> ...,POC Prod Report >> South Brintnell >> WB46,Well,Success,$min_speed = $rm+2\n$deviation2 = $min_speed- ...,"[rm=8A1E64DE-1213-4B41-AA9F-DC8BF87A6EC4, rw=C...",Seeq Data Lab,Seeq Data Lab,[7C166C13-9C0E-4C9D-8721-D1B4F894C13F] {Condit...,Success
11445,11445,FE9095DA-5B8C-47F7-8F23-2AF655F633D0,WB46 100/15C-01-081-23W4 SAM_PMP_EFF,CalculatedSignal,NaN,Well Structure,False,South Brintnell,WB46,SAM_PMP_EFF,...,POC Prod Report >> South Brintnell >> WB46 >> ...,POC Prod Report >> South Brintnell >> WB46,Well,Success,NaN,[signal=CDC47B4A-FB40-4BB8-8C46-A363BEABC733],Seeq Data Lab,Seeq Data Lab,[7C166C13-9C0E-4C9D-8721-D1B4F894C13F] {Signal...,Success
11446,11446,E0EC9A0A-63C0-478D-8BDE-412315D44B02,NaN,CalculatedSignal,NaN,NaN,NaN,NaN,NaN,NaN,...,POC Prod Report >> South Brintnell >> WB46 >> ...,POC Prod Report >> South Brintnell >> WB46,Well,Success,$pumpeff= $pmpeff > 100\n$rpmvar= $rpm > 10\n$...,"[pmpeff=FE9095DA-5B8C-47F7-8F23-2AF655F633D0, ...",Seeq Data Lab,Seeq Data Lab,[7C166C13-9C0E-4C9D-8721-D1B4F894C13F] {Signal...,Success
11447,11447,2512620B-3F28-4018-BA78-39C70489A8D2,NaN,Asset,NaN,NaN,NaN,NaN,NaN,NaN,...,POC Prod Report >> South Brintnell >> WB46 >> ...,POC Prod Report >> South Brintnell >> WB46,Well,Success,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[7C166C13-9C0E-4C9D-8721-D1B4F894C13F] {Asset}...,Success
